# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bredasdorp,-34.5322,20.0403,8.16,73,22,1.78,ZA,1722357760
1,1,makokou,0.5738,12.8642,24.42,68,83,1.10,GA,1722357761
2,2,codrington,-38.2667,141.9667,5.39,92,7,3.40,AU,1722357761
3,3,port elizabeth,-33.9180,25.5701,13.16,65,0,7.72,ZA,1722357761
4,4,toliara,-23.3500,43.6667,20.93,74,0,6.23,MG,1722357761


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:
%%capture --no-display

map_plot = city_data_df.hvplot.points(
    "Lng", "Lat",
    geo=True,
    size="Humidity",
    color="City",
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    hover_cols=["City", "Country", "Humidity"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# Use stipulations for 'ideal weather condition'
ideal_cities_df = city_data_df[
    (city_data_df["Max Temp"] > 21) &
    (city_data_df["Max Temp"] < 27) &
    (city_data_df["Wind Speed"] < 4.5) &
    (city_data_df["Cloudiness"] == 0)
]
# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
43,43,newman,37.3138,-121.0208,23.13,33,0,0.99,US,1722357766
107,107,tralee,52.2704,-9.7026,21.14,64,0,3.09,IE,1722357774
243,243,bemidji,47.4736,-94.8803,24.05,69,0,3.09,US,1722357790
440,440,heunghae,36.1125,129.3522,26.36,78,0,3.48,KR,1722357814
446,446,nova vicosa,-17.8919,-39.3719,26.92,63,0,4.44,BR,1722357815
492,492,sukumo,32.9333,132.7333,24.62,92,0,3.27,JP,1722357820
522,522,laguna,38.4210,-121.4238,24.43,66,0,2.57,US,1722357600
545,545,ambodifotatra,-16.9833,49.8500,23.76,61,0,1.43,MG,1722357826
573,573,tambopata,-12.7333,-69.1833,24.94,53,0,3.09,PE,1722358018
576,576,long beach,33.7670,-118.1892,25.10,74,0,2.06,US,1722357944


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
43,newman,US,37.3138,-121.0208,33,
107,tralee,IE,52.2704,-9.7026,64,
243,bemidji,US,47.4736,-94.8803,69,
440,heunghae,KR,36.1125,129.3522,78,
446,nova vicosa,BR,-17.8919,-39.3719,63,
492,sukumo,JP,32.9333,132.7333,92,
522,laguna,US,38.4210,-121.4238,66,
545,ambodifotatra,MG,-16.9833,49.8500,61,
573,tambopata,PE,-12.7333,-69.1833,53,
576,long beach,US,33.7670,-118.1892,74,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000 # 10,000 metres
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
newman - nearest hotel: No hotel found
tralee - nearest hotel: The Ashe Hotel
bemidji - nearest hotel: Hampton Inn & Suites
heunghae - nearest hotel: 호텔맨출장뷔페
nova vicosa - nearest hotel: Pousada Pontal da Barra
sukumo - nearest hotel: 上村ビジネスホテル
laguna - nearest hotel: Holiday Inn Express & Suites
ambodifotatra - nearest hotel: Les Palmiers
tambopata - nearest hotel: La Habana
long beach - nearest hotel: Westin


,City,Country,Lat,Lng,Humidity,Hotel Name
43,newman,US,37.3138,-121.0208,33,No hotel found
107,tralee,IE,52.2704,-9.7026,64,The Ashe Hotel
243,bemidji,US,47.4736,-94.8803,69,Hampton Inn & Suites
440,heunghae,KR,36.1125,129.3522,78,호텔맨출장뷔페
446,nova vicosa,BR,-17.8919,-39.3719,63,Pousada Pontal da Barra
492,sukumo,JP,32.9333,132.7333,92,上村ビジネスホテル
522,laguna,US,38.4210,-121.4238,66,Holiday Inn Express & Suites
545,ambodifotatra,MG,-16.9833,49.8500,61,Les Palmiers
573,tambopata,PE,-12.7333,-69.1833,53,La Habana
576,long beach,US,33.7670,-118.1892,74,Westin


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng", "Lat",
    geo=True,
    size="Humidity",
    color="City",
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    hover_cols=["City", "Country", "Humidity", "Hotel Name"]
)


# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)